Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
                            plot_confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import tarfile

# I. Wrangle Data

In [ ]:
# Download data
def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://lambdaschool-ds-instruction.s3.amazonaws.com/datasets%3Ainstacart_2017_05_01.tar.gz')

Downloaded datasets%3Ainstacart_2017_05_01.tar.gz


In [ ]:
# # Uncompress data
tarfile.open('datasets%3Ainstacart_2017_05_01.tar.gz').extractall()

In [ ]:
# Print the csv filenames
pattern = 'instacart_2017_05_01/*.csv'

for filename in glob(pattern):
    print(filename)

instacart_2017_05_01/departments.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/products.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/order_products__train.csv


**Before you start,** load each of the above `.csv` files into its own DataFrame.

In [ ]:
orders = pd.read_csv('instacart_2017_05_01/orders.csv')
order_products_train = pd.read_csv('instacart_2017_05_01/order_products__train.csv')
order_products_prior = pd.read_csv('instacart_2017_05_01/order_products__prior.csv')
products = pd.read_csv('instacart_2017_05_01/products.csv')

## I.a. Warm-up Questions

What information is contained in the column `orders['eval_set']`?

In [ ]:
orders['eval_set'].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

The first row of `orders['order_id']` is `2539329`. Where can we find the items that were included in that order?

In [ ]:
mask = (order_products_prior['order_id']==2539329)
order_products_prior[mask]

,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


The first row of `order_products__prior['product_id']` is `33120`. What is the name of that product?

In [ ]:
mask = products['product_id']==33120
products[mask]

,product_id,product_name,aisle_id,department_id
33119,33120,Organic Egg Whites,86,16


## I.b. Define Our Machine Learning Problem

- We want predict whether or not a customer will purchase a specific item (or our choosing).

In [ ]:
order_products_prior['product_id'].value_counts().head(10)

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
47766    176815
47626    152657
16797    142951
26209    140627
27845    137905
Name: product_id, dtype: int64

In [ ]:
mask = products['product_id']==24852
products[mask]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


## I.c. Create Feature Matrix and Target Vector

Our **feature matrix** will be all the `'train'` rows from `orders`.

In [ ]:
mask = orders['eval_set']=='train'

X=orders[mask].copy()
X.drop(columns='eval_set',inplace=True)

In [ ]:
X

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,11,4,8,14.0
25,1492625,2,15,1,11,30.0
49,2196797,5,5,0,11,6.0
74,525192,7,21,2,11,6.0
78,880375,8,4,1,14,10.0
...,...,...,...,...,...,...
3420838,2585586,206199,20,2,16,30.0
3420862,943915,206200,24,6,19,6.0
3420924,2371631,206203,6,4,19,30.0
3420933,1716008,206205,4,1,16,10.0


Our **target vector** will be whether or not each order in `X` contains the item we've chosen above.

In [ ]:
order_products_train['is_banana']=order_products_train['product_id']==24852

In [ ]:
order_products_train['is_banana']

0          False
1          False
2          False
3          False
4          False
           ...  
1384612    False
1384613    False
1384614    False
1384615    False
1384616    False
Name: is_banana, Length: 1384617, dtype: bool

In [ ]:
order_products_train

,order_id,product_id,add_to_cart_order,reordered,is_banana
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False
...,...,...,...,...,...
1384612,3421063,14233,3,1,False
1384613,3421063,35548,4,1,False
1384614,3421070,35951,1,1,False
1384615,3421070,16953,2,1,False


In [ ]:
banana_orders = order_products_train[order_products_train['is_banana']]['order_id']

In [ ]:
X

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,11,4,8,14.0
25,1492625,2,15,1,11,30.0
49,2196797,5,5,0,11,6.0
74,525192,7,21,2,11,6.0
78,880375,8,4,1,14,10.0
...,...,...,...,...,...,...
3420838,2585586,206199,20,2,16,30.0
3420862,943915,206200,24,6,19,6.0
3420924,2371631,206203,6,4,19,30.0
3420933,1716008,206205,4,1,16,10.0


In [ ]:
X['includes_bananas']=X['order_id']

In [ ]:
X['includes_bananas']=X['order_id'].isin(banana_orders).astype(int)

# II. Split Data

In [ ]:
X

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,includes_bananas
10,1187899,1,11,4,8,14.0,0
25,1492625,2,15,1,11,30.0,1
49,2196797,5,5,0,11,6.0,0
74,525192,7,21,2,11,6.0,0
78,880375,8,4,1,14,10.0,0
...,...,...,...,...,...,...,...
3420838,2585586,206199,20,2,16,30.0,0
3420862,943915,206200,24,6,19,6.0,0
3420924,2371631,206203,6,4,19,30.0,0
3420933,1716008,206205,4,1,16,10.0,1


## Feature Engineering

In [ ]:
X['morning']=(X['order_hour_of_day']>11).astype(int)

In [ ]:
n_items_per_order=order_products_train.groupby('order_id')['product_id'].count().rename('number_of_items')

In [ ]:
print(n_items_per_order)

order_id
1           8
36          8
38          9
96          7
98         49
           ..
3421049     6
3421056     5
3421058     8
3421063     4
3421070     3
Name: number_of_items, Length: 131209, dtype: int64


In [ ]:
X= X.merge(n_items_per_order,
        how='left',
        left_on='order_id',
        right_index=True)

In [ ]:
target = 'includes_bananas'
y = X[target]
X = X.drop(columns=[target,'order_number','order_id','user_id'])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# III. Establish Baseline

In [ ]:
print('Baseline accuracy:', y_train.value_counts(normalize=True).max())

Baseline accuracy: 0.8569931502281669


# IV. Build Model

In [ ]:
model_rf = RandomForestClassifier(random_state=42,n_jobs=-1)
model_rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

# V. Check Metrics

In [ ]:
print('Training Accuracy:', model_rf.score(X_train,y_train))
print('Validation Accuracy:', model_rf.score(X_val,y_val))

Training Accuracy: 0.9145826783655816
Validation Accuracy: 0.826499504610929


In [ ]:
thresholds=np.arrange[0.05,1,0.5]

accuracy=[]
recall=[]
precision=[]

y_pred_proba > 0.7 class 1
<0 class 0